# BASE CODE

In [ ]:
# Importing the libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Extra libraries
import json
import googlemaps
from datetime import datetime

with open('creds.json') as f:
    api_key = json.load(f)

# API KEY    
## api_key['GOOGLE_MAPS_API_KEYS']

In [ ]:
df = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv") # OR USE searchData/limitData
df = df.drop_duplicates()
print(df.dtypes)
df.head(2)

In [ ]:
df['year'] = df['month'].apply(lambda x: x.split('-')[0]).astype(int)
df['month'] = df['month'].apply(lambda x: x.split('-')[1]).astype(int)

# Convert remaining lease from years to months
df['month_remaining_lease'] = df['remaining_lease'].apply(lambda x: int(x.split(' ')[0]) * 12 + int(x.split(' ')[2]) if len(x.split(' ')) > 2 else int(x.split(' ')[0]) * 12)

# Split storey range into min and max
df['storey_range_low'] = df['storey_range'].apply(lambda x: int(x.split('TO')[0]))
df['storey_range_high'] = df['storey_range'].apply(lambda x: int(x.split('TO')[1]))

# Convert categorical columns to numerical
df['flat_model'] = df['flat_model'].astype('category').cat.codes

df[['month', 'year', 'town', 'flat_type', 'floor_area_sqm', 'lease_commence_date', 'month_remaining_lease', 'flat_model', 'storey_range_low', 'storey_range_high', 'resale_price']].head(5)

# ADD ON COLUMNS CODE

## 1. Get walking distance to nearest MRT station

In [ ]:
gmaps = googlemaps.Client(key=api_key['GOOGLE_MAPS_API_KEYS'])

def get_nearest_stn(addr):
    # Request directions via public transit
    # Returns Start, End, Total distance

    coords = gmaps.geocode(f"{addr}, Singapore")
    
    lat = coords[0]['geometry']['location']['lat']
    lng = coords[0]['geometry']['location']['lng']

    stn_found = False
    radius = 100

    while not stn_found:
        nearest_station = gmaps.places('MRT', location=(lat, lng), radius=radius)
        if len(nearest_station['results']) > 0:
            stn_found = True
        else:
            radius += 100

    directions_result = gmaps.directions(f"{addr}, Singapore",
                                     f"{nearest_station['results'][0]['formatted_address']}",
                                     mode="walking",
                                     departure_time=datetime.now())[0]
    
    #print("START:", directions_result['legs'][0]['start_address'])
    #print("END:", directions_result['legs'][0]['end_address'])
    #print("DISTANCE:", directions_result['legs'][0]['distance']['text'])
    
    return directions_result['legs'][0]['distance']['text'].split(' ')[0]

In [ ]:
for index, row in df.iterrows():
    start, end, dist = get_nearest_stn(f"{row['block']} {row['street_name']}, Singapore")
    df.loc[index, 'dist_to_nearest_stn_walking'] = dist

## 2. Get walking distance to nearest mall

## 3. Get walking distance to nearest park